In [1]:
%pip install noaa-sdk
%pip install meteostat
#%pip install xgboost
#%pip install imblearn
#%pip install scikit-learn
#%pip install torch
#%pip install typing_extensions

In [2]:
import pandas as pd
import numpy as np
from noaa_sdk import NOAA
from meteostat import Point, Hourly
from datetime import datetime, timedelta, date
import pytz
import re
import requests
import pickle
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from imblearn.under_sampling import RandomUnderSampler
import math
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
import networkx as nx
from sklearn.preprocessing import OneHotEncoder

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
import os
import sys
# Get the current working directory
dir_path = '/content/gdrive/My Drive/JetLagged/data'
sys.path.append(dir_path)

In [5]:
# The commented code below is the preprocessing task, it takes too much time so I ran the code in local and loading the completed csv here

# schedules_df = pd.read_csv("{}/OTP_DETAIL_UC_LARGE_HUB_1123971.csv".format(dir_path))
# schedules_df = schedules_df.dropna(subset=['Origin Airport Code'])
# airport_info_df = pd.read_csv("{}/airports_info.csv".format(dir_path))
# airport_info_df = airport_info_df[['code', 'latitude','longitude', 'elevation','time_zone']]
# print(schedules_df.columns)
# print(airport_info_df.columns)

flight_weather = pd.read_csv("{}/processed/flight_weather_UTC.csv".format(dir_path), low_memory=False)
print(flight_weather.dtypes)

Unnamed: 0                       int64
Date                            object
Destination Airport Code        object
Origin Airport Code             object
Direction                       object
Departure Count                  int64
Departure Delay                  int64
Departure Delay > 15 Min        object
Arrival Delay                    int64
Arrival Delay > 15 Min          object
Cancelled Flight                object
Scheduled Departure Time        object
Actual Departure Time           object
Arrival Delay Air System         int64
Arrival Delay Weather            int64
Arrival Delay Group Code       float64
Aircraft Type                   object
Mkt. Carrier Code               object
Op. Carrier Code                object
Actual Arrival Time             object
Scheduled Arrival Time          object
Departure Delay Group Code     float64
Arrival Delay Carrier            int64
Arrival Delay Late Aircraft      int64
Arrival Delay Security           int64
code                     

In [ ]:
# flight_df = pd.merge(schedules_df, airport_info_df, left_on='Origin Airport Code', right_on='code')
# print(flight_df.head(4))

In [ ]:
# def convert_to_utc(row):
#   dep_time = datetime.strptime(row['Scheduled Departure Time'], '%H:%M').time()
#   date = pd.to_datetime(row['Date']).date()
#   local_dt = datetime.combine(date, dep_time)
#   local_tz = pytz.timezone(row['time_zone'])
#   local_dt = local_tz.localize(local_dt)
#   utc_dt = local_dt.astimezone(pytz.UTC)
#   return utc_dt

In [ ]:
# flight_weather["Scheduled Departure UTC"] = flight_weather.apply(convert_to_utc, axis=1)
# # print(flight_weather)

In [ ]:
# def fetch_weather(row):
#     # target_time = row['hour_bin'].tz_convert(None)
#     start = row['hour_bin']
#     end = start + pd.Timedelta(hours=1)

#     location = Point(row['latitude'], row['longitude'], row['elevation'])
#     hourly_data = Hourly(location, start, end).fetch()
#     hourly_data['Origin Airport Code'] = row['Origin Airport Code']
#     hourly_data['hour_bin'] = row['hour_bin']

#     return hourly_data

In [ ]:

# flight_df['hour_bin'] = flight_df['Scheduled Departure UTC'].dt.floor('h').dt.tz_convert(None)
# unique_times = flight_df[['Origin Airport Code', 'latitude', 'elevation','longitude', 'hour_bin']].drop_duplicates()

# weather_data= []
# weather_data = unique_times.apply(fetch_weather, axis=1).tolist()
# weather_df = pd.concat(weather_data).reset_index()
# print(weather_df)

In [ ]:

# flight_weather = pd.merge(flight_df, weather_df, left_on=['Origin Airport Code', 'hour_bin'],
#                      right_on=['Origin Airport Code', 'time'], how='left')

# Data Exploration and Preprocessing

In [ ]:
# row sample
flight_weather.iloc[0]

,0
Unnamed: 0,0
Date,01/01/2023
Destination Airport Code,AUS
Origin Airport Code,ATL
Direction,Outbound
Departure Count,1
Departure Delay,0
Departure Delay > 15 Min,No
Arrival Delay,0
Arrival Delay > 15 Min,No


In [ ]:
print("Number of total data frame rows: " + str(len(flight_weather)))

Number of total data frame rows: 4015685


In [ ]:
flight_weather_sub = flight_weather.dropna(subset=['temp'])
print("Number of rows present in dataset where there is populated weather data: " + str(len(flight_weather_sub)))

Number of rows present in dataset where there is populated weather data: 2859569


In [ ]:
flight_weather_sub.describe()

,Unnamed: 0,Departure Count,Departure Delay,Arrival Delay,Arrival Delay Air System,Arrival Delay Weather,Arrival Delay Group Code,Departure Delay Group Code,Arrival Delay Carrier,Arrival Delay Late Aircraft,...,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,2.859569e+06,2859569.0,2.859569e+06,2.859569e+06,2.859569e+06,2.859569e+06,2.809958e+06,2.819867e+06,2.859569e+06,2.859569e+06,...,2.859569e+06,2.859569e+06,2.859569e+06,0.0,2.859569e+06,2.859569e+06,0.0,2.859569e+06,0.0,2.859569e+06
mean,2.130137e+06,1.0,1.765674e+01,1.734527e+01,3.520227e+00,6.846039e-01,-2.633990e-02,3.078514e-01,5.841739e+00,6.253631e+00,...,1.081396e+01,6.574508e+01,1.358373e-01,NaN,1.768747e+02,1.352005e+01,NaN,1.016111e+03,NaN,3.387448e+00
std,1.109403e+06,0.0,5.761902e+01,5.683850e+01,1.733368e+01,1.232621e+01,2.675269e+00,2.504881e+00,3.768604e+01,3.265944e+01,...,8.959610e+00,1.883642e+01,1.089180e+00,NaN,1.116884e+02,8.111955e+00,NaN,6.286164e+00,NaN,2.711798e+00
min,1.522560e+05,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00,...,-3.510000e+01,7.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,9.806000e+02,NaN,1.000000e+00
25%,1.355644e+06,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,...,4.800000e+00,5.200000e+01,0.000000e+00,NaN,8.000000e+01,7.600000e+00,NaN,1.012500e+03,NaN,2.000000e+00
50%,2.178264e+06,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,...,1.190000e+01,6.700000e+01,0.000000e+00,NaN,1.900000e+02,1.300000e+01,NaN,1.015900e+03,NaN,3.000000e+00
75%,3.142408e+06,1.0,1.100000e+01,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.810000e+01,8.100000e+01,0.000000e+00,NaN,2.700000e+02,1.840000e+01,NaN,1.019900e+03,NaN,3.000000e+00
max,4.015684e+06,1.0,2.905000e+03,2.912000e+03,1.409000e+03,1.440000e+03,1.200000e+01,1.200000e+01,2.827000e+03,2.557000e+03,...,2.810000e+01,1.000000e+02,6.380000e+01,NaN,3.600000e+02,7.960000e+01,NaN,1.041800e+03,NaN,2.500000e+01


In [ ]:
def bucket_direction(direction):
    # Define cardinal direction buckets
    buckets = np.arange(0, 360, 22.5)
    closest_bucket = min(buckets, key=lambda x: abs(x - direction))
    return closest_bucket

These cells flag certain dates based on the criteria outlined here:

https://www.oag.com/blog/busiest-days-for-air-travel-2009-2024

https://www.nerdwallet.com/article/travel/busiest-days-to-fly-during-winter-holidays-2023

In [ ]:
# returns true if the given day is a friday, saturday, or sunday
def is_travel_day(d) :
    return d.weekday() >= 4

# returns true if the given day is within the first two weeks of august
def is_summer_travel_season(d) :
    if d.month != 8:
        return False

    # count number of fridays which have occurred in august
    fri_count = sum(1 for day in range(1, d.day + 1) if date(d.year, 8, day).weekday() == 4)
    beginning_august = d.month == 8 and fri_count in [1, 2]
    jul = d.month == 7

    return beginning_august or jul

def thanksgiving_travel(d):
    # Find Thanksgiving (4th Thursday of November)
    first_day_of_november = date(d.year, 11, 1)
    first_thursday = (3 - first_day_of_november.weekday() + 7) % 7
    thanksgiving = first_day_of_november + timedelta(days=first_thursday + 21)

    # Map each travel day to its calculated date
    travel_days = [
        thanksgiving + timedelta(days=3),  # Sunday after Thanksgiving
        thanksgiving - timedelta(days=1),  # Wednesday before Thanksgiving
        thanksgiving - timedelta(days=6),  # Friday before Thanksgiving
        thanksgiving + timedelta(days=4),  # Monday after Thanksgiving (Cyber Monday)
        thanksgiving + timedelta(days=2),  # Saturday after Thanksgiving
        thanksgiving - timedelta(days=4),  # Sunday before Thanksgiving
        thanksgiving - timedelta(days=2),  # Tuesday before Thanksgiving
        thanksgiving - timedelta(days=7),  # Thursday before Thanksgiving
        thanksgiving - timedelta(days=3),  # Monday before Thanksgiving
        thanksgiving - timedelta(days=5),  # Saturday before Thanksgiving
        thanksgiving + timedelta(days=7),  # Thursday after Thanksgiving
        thanksgiving + timedelta(days=5),  # Tuesday after Thanksgiving (Giving Tuesday)
        thanksgiving + timedelta(days=1),  # Friday after Thanksgiving (Black Friday)
        thanksgiving + timedelta(days=6),  # Wednesday after Thanksgiving
        thanksgiving,                     # Thanksgiving Day (Thursday)
    ]

    # Return True if the date matches any busy travel day
    return d in travel_days

def holiday_travel(d):
    holiday_travel = [
        (12, 27),  # December 27
        (12, 30),  # December 30
        (12, 20),  # December 20
        (12, 26),  # December 26
        (12, 23),  # December 23
        (12, 22),  # December 22
        (12, 19),  # December 19
        (12, 21),  # December 21
        (12, 29),  # December 29
        (12, 28),  # December 28
        (12, 18),  # December 18
        (1, 1),    # January 1 (New Year’s Day)
        (12, 31),  # December 31 (New Year’s Eve)
        (12, 24),  # December 24 (Christmas Eve)
        (12, 25),  # December 25 (Christmas Day)
    ]

    # Check if the input date's month and day match any of the dates
    return (d.month, d.day) in holiday_travel


In [ ]:
flight_weather_sub['Departure Delay > 15 Min'] = flight_weather_sub['Departure Delay > 15 Min'].map({'Yes': 1, 'No': 0})
flight_weather_sub['Arrival Delay > 15 Min'] = flight_weather_sub['Arrival Delay > 15 Min'].map({'Yes': 1, 'No': 0})
flight_weather_sub['Cancelled Flight'] = flight_weather_sub['Cancelled Flight'].map({'Yes': 1, 'No': 0})
flight_weather_sub['gust'] = flight_weather_sub['wpgt'].fillna(flight_weather_sub['wspd'])
flight_weather_sub['precip_chance'] = flight_weather_sub['prcp'].apply(lambda x: 100 if not x == 0 else 0)
flight_weather_sub['wdir'] = flight_weather_sub['wdir'].apply(lambda x: bucket_direction(x))
flight_weather_sub['weekend'] = pd.to_datetime(flight_weather_sub['Date']).apply(lambda x: 1 if is_travel_day(x) else 0)
flight_weather_sub['summer_travel'] = pd.to_datetime(flight_weather_sub['Date']).apply(lambda x: 1 if is_summer_travel_season(x) else 0)
flight_weather_sub['thanksgiving'] = pd.to_datetime(flight_weather_sub['Date']).apply(lambda x: 1 if thanksgiving_travel(x) else 0)
flight_weather_sub['holiday'] = pd.to_datetime(flight_weather_sub['Date']).apply(lambda x: 1 if holiday_travel(x) else 0)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

Create a new dataframe of only flights that were not cancelled

In [ ]:
non_cancelled_flights = flight_weather_sub[flight_weather_sub['Cancelled Flight'] == 0]

Extremely skewed classification, so let's take all delayed flights then an equal number of flights not delayed as our sample

In [ ]:
# Filter relevant columns for the network
non_cancelled_flights_filtered = non_cancelled_flights[['Origin Airport Code', 'Destination Airport Code', 'Departure Count']]

# Create a directed graph
G = nx.DiGraph()

# Add weighted edges based on Departure Count
for _, row in non_cancelled_flights_filtered.iterrows():
    origin = row['Origin Airport Code']
    destination = row['Destination Airport Code']
    weight = row['Departure Count']
    if G.has_edge(origin, destination):
        # If the edge already exists, update the weight
        G[origin][destination]['weight'] += weight
    else:
        # Add a new edge with the weight
        G.add_edge(origin, destination, weight=weight)

# Calculate centrality measures
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, normalized=True, weight='weight')

# Add centrality features to the dataframe
non_cancelled_flights['Degree Centrality Arrival'] = non_cancelled_flights['Destination Airport Code'].map(degree_centrality)
non_cancelled_flights['Closeness Centrality Arrival'] = non_cancelled_flights['Destination Airport Code'].map(closeness_centrality)
non_cancelled_flights['Betweenness Centrality Arrival'] = non_cancelled_flights['Destination Airport Code'].map(betweenness_centrality)

non_cancelled_flights['Degree Centrality Departure'] = non_cancelled_flights['Origin Airport Code'].map(degree_centrality)
non_cancelled_flights['Closeness Centrality Departure'] = non_cancelled_flights['Origin Airport Code'].map(closeness_centrality)
non_cancelled_flights['Betweenness Centrality Departure'] = non_cancelled_flights['Origin Airport Code'].map(betweenness_centrality)

# Save the network
pickle.dump(G, open('{}/flight_network.pickle'.format(dir_path), 'wb'))

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [ ]:
# Add Quarter, Month, and Day of Week
non_cancelled_flights['Date'] = pd.to_datetime(non_cancelled_flights['Date'], format='%m/%d/%Y')
non_cancelled_flights['Quarter'] = non_cancelled_flights['Date'].dt.quarter
non_cancelled_flights['Month'] = non_cancelled_flights['Date'].dt.month
non_cancelled_flights['Day of Week'] = non_cancelled_flights['Date'].dt.dayofweek

# One-hot encode Month and Day of Week
encoder = OneHotEncoder(sparse_output=False)
month_encoded = encoder.fit_transform(non_cancelled_flights[['Month']])
day_of_week_encoded = encoder.fit_transform(non_cancelled_flights[['Day of Week']])

# Add encoded features to the dataframe
month_columns = [f'Month_{i+1}' for i in range(month_encoded.shape[1])]
day_columns = [f'Day_{i}' for i in range(day_of_week_encoded.shape[1])]

non_cancelled_flights[month_columns] = month_encoded
non_cancelled_flights[day_columns] = day_of_week_encoded

# Add Departure and Arrival Time in 1-hour buckets
non_cancelled_flights['Scheduled Departure Time'] = pd.to_datetime(non_cancelled_flights['Scheduled Departure Time'], format='%H:%M')
non_cancelled_flights['Scheduled Arrival Time'] = pd.to_datetime(non_cancelled_flights['Scheduled Arrival Time'], format='%H:%M')

non_cancelled_flights['Departure Hour Bucket'] = non_cancelled_flights['Scheduled Departure Time'].dt.hour
non_cancelled_flights['Arrival Hour Bucket'] = non_cancelled_flights['Scheduled Arrival Time'].dt.hour

# Final dataframe with all added features
print(non_cancelled_flights.head())
print(non_cancelled_flights.columns)
print(len(non_cancelled_flights.columns))

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

        Unnamed: 0       Date Destination Airport Code Origin Airport Code  \
152256      152256 2023-01-01                      ATL                 AUS   
152257      152257 2023-01-01                      ATL                 AUS   
152258      152258 2023-01-01                      ATL                 AUS   
152259      152259 2023-01-01                      ATL                 AUS   
152260      152260 2023-01-01                      ATL                 AUS   

       Direction  Departure Count  Departure Delay  Departure Delay > 15 Min  \
152256  Outbound                1                8                         0   
152257  Outbound                1                0                         0   
152258  Outbound                1                0                         0   
152259  Outbound                1                0                         0   
152260  Outbound                1                0                         0   

        Arrival Delay  Arrival Delay > 15 Min  ...

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 50-50 data

In [ ]:
features = ['Betweenness Centrality Arrival', 'Betweenness Centrality Departure', 'Closeness Centrality Arrival', 'Closeness Centrality Departure','Degree Centrality Arrival', 'Degree Centrality Departure', 'Quarter', 'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',  'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Day_0', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6',
'Departure Hour Bucket', 'Arrival Hour Bucket', 'elevation', 'temp', 'dwpt', 'rhum', 'precip_chance', 'wdir', 'wspd', 'gust']

unused_features = ['weekend', 'summer_travel', 'thanksgiving', 'holiday']

target = 'Departure Delay > 15 Min'
print(str(non_cancelled_flights[target].value_counts()))

delayed = non_cancelled_flights[non_cancelled_flights[target] == 1]
non_delayed = non_cancelled_flights[non_cancelled_flights[target] == 0]
# find the number of delays
num_delays = len(delayed)
sampled_nondelay = non_delayed.sample(n=num_delays, random_state=74)

# recombine and shuffle
balanced_sample = pd.concat([delayed, sampled_nondelay], ignore_index=True)
balanced_sample = balanced_sample.sample(frac=1, random_state=89).reset_index(drop=True)

balanced_sample.describe()

0    2180679
1     637232
Name: Departure Delay > 15 Min, dtype: int64


,Unnamed: 0,Departure Count,Departure Delay,Departure Delay > 15 Min,Arrival Delay,Arrival Delay > 15 Min,Cancelled Flight,Arrival Delay Air System,Arrival Delay Weather,Arrival Delay Group Code,...,Month_12,Day_0,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Departure Hour Bucket,Arrival Hour Bucket
count,1.274464e+06,1274464.0,1.274464e+06,1274464.0,1.274464e+06,1.274464e+06,1274464.0,1.274464e+06,1.274464e+06,1.269949e+06,...,1.274464e+06,1.274464e+06,1.274464e+06,1.274464e+06,1.274464e+06,1.274464e+06,1.274464e+06,1.274464e+06,1.274464e+06,1.274464e+06
mean,2.119938e+06,1.0,3.773961e+01,0.5,3.547497e+01,4.209432e-01,0.0,5.906378e+00,1.524449e+00,1.162990e+00,...,7.930863e-02,1.476872e-01,1.353063e-01,1.389031e-01,1.467488e-01,1.507991e-01,1.300311e-01,1.505245e-01,1.372560e+01,1.521797e+01
std,1.101010e+06,0.0,8.159670e+01,0.5,8.102854e+01,4.937107e-01,0.0,2.430169e+01,1.842366e+01,3.464032e+00,...,2.702200e-01,3.547898e-01,3.420506e-01,3.458455e-01,3.538554e-01,3.578531e-01,3.363378e-01,3.575849e-01,4.931444e+00,5.507390e+00
min,1.522630e+05,1.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.360968e+06,1.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00,1.100000e+01
50%,2.164374e+06,1.0,1.450000e+01,0.5,7.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.400000e+01,1.600000e+01
75%,2.916368e+06,1.0,4.200000e+01,1.0,4.000000e+01,1.000000e+00,0.0,0.000000e+00,0.000000e+00,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.800000e+01,2.000000e+01
max,4.015684e+06,1.0,2.905000e+03,1.0,2.912000e+03,1.000000e+00,0.0,1.409000e+03,1.440000e+03,1.200000e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.300000e+01,2.300000e+01


Now we'll scale these inputs and save the scaler for future use

In [ ]:
x_unscaled = balanced_sample[features]
y = balanced_sample[target]

scaler = MinMaxScaler()
scaler.fit(x_unscaled)
x = scaler.transform(x_unscaled)
path = "{}/../models/".format(dir_path)
joblib.dump(scaler, '{}/scaler.pkl'.format(path))

['Z:/VT/F24/urban_computing/JetLagged/data/../models//scaler.pkl']

## SMOTE Data

In [ ]:
# Split the data into features and target
X = non_cancelled_flights[features]
print(X.head())
y = non_cancelled_flights[target]
print(y.head())

# Initialize SMOTE with the same random state for reproducibility
smote = SMOTE(random_state=42, sampling_strategy='auto')

# Apply SMOTE to oversample the minority class
X_smote, y_smote = smote.fit_resample(X, y)

# Combine oversampled data back into a DataFrame
smote_df = pd.DataFrame(X_smote, columns=features)
smote_df[target] = y_smote

# Balance the dataset size to match the original size
# Calculate how many rows to sample from the oversampled dataset
sample_size = len(non_cancelled_flights)

# Shuffle and downsample to the original size
SMOTE_balanced_sample = smote_df.sample(n=sample_size, random_state=89)
SMOTE_balanced_sample = shuffle(SMOTE_balanced_sample, random_state=89).reset_index(drop=True)

# Describe the balanced dataset
print(SMOTE_balanced_sample[target].value_counts())
SMOTE_balanced_sample.describe()


        Betweenness Centrality Arrival  Betweenness Centrality Departure  \
152256                             0.0                          0.062069   
152257                             0.0                          0.062069   
152258                             0.0                          0.062069   
152259                             0.0                          0.062069   
152260                             0.0                          0.062069   

        Closeness Centrality Arrival  Closeness Centrality Departure  \
152256                      0.633333                        0.568421   
152257                      0.633333                        0.568421   
152258                      0.633333                        0.568421   
152259                      0.633333                        0.568421   
152260                      0.633333                        0.568421   

        Degree Centrality Arrival  Degree Centrality Departure  Quarter  \
152256                   0.633333  

,Betweenness Centrality Arrival,Betweenness Centrality Departure,Closeness Centrality Arrival,Closeness Centrality Departure,Degree Centrality Arrival,Degree Centrality Departure,Quarter,Month_1,Month_2,Month_3,...,Arrival Hour Bucket,elevation,temp,dwpt,rhum,precip_chance,wdir,wspd,gust,Departure Delay > 15 Min
count,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,...,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06,2.817911e+06
mean,2.064111e-02,3.765743e-02,5.791821e-01,5.411214e-01,1.084199e+00,1.448767e+00,2.478928e+00,8.084654e-02,7.242702e-02,8.929951e-02,...,1.516058e+01,9.260228e+01,1.851464e+01,1.126813e+01,6.575773e+01,9.508711e+00,1.768204e+02,1.382864e+01,1.382864e+01,5.002433e-01
std,3.976359e-02,4.695846e-02,5.589740e-02,5.265382e-02,4.110768e-01,1.510912e-01,1.092023e+00,2.713593e-01,2.580078e-01,2.839153e-01,...,5.494787e+00,1.085165e+02,8.534309e+00,8.970655e+00,1.884177e+01,2.933353e+01,1.092992e+02,8.154236e+00,8.154236e+00,5.000000e-01
min,0.000000e+00,0.000000e+00,4.153846e-01,4.153846e-01,5.333333e-01,9.666667e-01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,1.300000e+01,-2.290000e+01,-3.510000e+01,7.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,3.448276e-03,5.424702e-01,5.142857e-01,6.333333e-01,1.366667e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.100000e+01,2.900000e+01,1.260000e+01,5.400000e+00,5.200000e+01,0.000000e+00,9.000000e+01,7.600000e+00,7.600000e+00,0.000000e+00
50%,0.000000e+00,1.379310e-02,6.000000e-01,5.684211e-01,1.200000e+00,1.500000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.600000e+01,4.900000e+01,1.910000e+01,1.240000e+01,6.700000e+01,0.000000e+00,1.800000e+02,1.300000e+01,1.300000e+01,1.000000e+00
75%,2.298851e-02,6.091954e-02,6.333333e-01,5.684211e-01,1.500000e+00,1.533333e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e+01,1.250000e+02,2.530000e+01,1.870000e+01,8.100000e+01,0.000000e+00,2.700000e+02,1.840000e+01,1.840000e+01,1.000000e+00
max,2.793103e-01,2.793103e-01,6.333333e-01,6.000000e-01,1.600000e+00,1.600000e+00,4.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,2.300000e+01,5.050000e+02,4.250000e+01,2.810000e+01,1.000000e+02,1.000000e+02,3.375000e+02,7.960000e+01,7.960000e+01,1.000000e+00


In [ ]:
X_smote = SMOTE_balanced_sample[features]
y_smote = SMOTE_balanced_sample[target]

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_smote)

# Split the dataset into training and testing sets
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_scaled, y_smote, test_size=0.2, random_state=45)
path = "{}/../models/".format(dir_path)
joblib.dump(scaler, '{}/scaler_smote.pkl'.format(path))

['Z:/VT/F24/urban_computing/JetLagged/data/../models//scaler_smote.pkl']

# Training the Models
Skip this and load the models if you want to execute predictions!
## Decision Tree Classifier

In [ ]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Initialize and train the Decision Tree Classifier
tree = DecisionTreeClassifier(random_state=42) # Adjust parameters like max_depth, min_samples_split etc. as needed
tree.fit(X_train, y_train)

# Make predictions on the test set
y_pred = tree.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# save model
path = "{}/../models/".format(dir_path)
with open("{}/decision_tree.pkl".format(path), "wb") as file:
    pickle.dump(tree, file)

print(f"Decision Tree Accuracy: {accuracy}")
print(f"Decision Tree Precision: {precision}")
print(f"Decision Tree Recall: {recall}")
print(f"Decision Tree F1 Score: {f1}")
print(str(confusion_matrix(y_test, y_pred)))

Decision Tree Accuracy: 0.7660861616442978
Decision Tree Precision: 0.7460954541488526
Decision Tree Recall: 0.8059694458626242
Decision Tree F1 Score: 0.7748775708239097
[[ 92658  34920]
 [ 24703 102612]]


In [ ]:
# Initialize and train the Decision Tree Classifier
tree_smote = DecisionTreeClassifier(random_state=42) # Adjust parameters like max_depth, min_samples_split etc. as needed
tree_smote.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred = tree_smote.predict(X_test_smote)

# Evaluate the model
accuracy = accuracy_score(y_test_smote, y_pred)
precision = precision_score(y_test_smote, y_pred)
recall = recall_score(y_test_smote, y_pred)
f1 = f1_score(y_test_smote, y_pred)

# save model
path = "{}/../models/".format(dir_path)
with open("{}/decision_tree_smote.pkl".format(path), "wb") as file:
    pickle.dump(tree_smote, file)

print(f"Decision Tree Accuracy: {accuracy}")
print(f"Decision Tree Precision: {precision}")
print(f"Decision Tree Recall: {recall}")
print(f"Decision Tree F1 Score: {f1}")
print(str(confusion_matrix(y_test_smote, y_pred)))

Decision Tree Accuracy: 0.8747070085506483
Decision Tree Precision: 0.8684248189196265
Decision Tree Recall: 0.8831389198514743
Decision Tree F1 Score: 0.8757200661061607
[[244188  37693]
 [ 32920 248782]]


## Logistic Regression

In [ ]:
# Initialize and train the Logistic Regression model
logreg = LogisticRegression(random_state=42, max_iter=1000) # Increase max_iter if needed
logreg.fit(X_train, y_train)

# Make predictions on the test set
y_pred_logreg = logreg.predict(X_test)

# Evaluate the model
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
precision_logreg = precision_score(y_test, y_pred_logreg)
recall_logreg = recall_score(y_test, y_pred_logreg)
f1_logreg = f1_score(y_test, y_pred_logreg)

# save model
#with open("{}/logistic.pkl".format(path), "wb") as file:
#    pickle.dump(logreg, file)

print(f"Logistic Regression Accuracy: {accuracy_logreg}")
print(f"Logistic Regression Precision: {precision_logreg}")
print(f"Logistic Regression Recall: {recall_logreg}")
print(f"Logistic Regression F1 Score: {f1_logreg}")
print(str(confusion_matrix(y_test, y_pred_logreg)))

Logistic Regression Accuracy: 0.6248504274342567
Logistic Regression Precision: 0.6214029603751035
Logistic Regression Recall: 0.6370655460864785
Logistic Regression F1 Score: 0.6291367869096607
[[78162 49416]
 [46207 81108]]


In [ ]:
# Initialize and train the Logistic Regression model
logreg_smote = LogisticRegression(random_state=42, max_iter=1000) # Increase max_iter if needed
logreg_smote.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred_logreg = logreg_smote.predict(X_test_smote)

# Evaluate the model
accuracy_logreg = accuracy_score(y_test_smote, y_pred_logreg)
precision_logreg = precision_score(y_test_smote, y_pred_logreg)
recall_logreg = recall_score(y_test_smote, y_pred_logreg)
f1_logreg = f1_score(y_test_smote, y_pred_logreg)

# save model
with open("{}/logistic_smote.pkl".format(path), "wb") as file:
    pickle.dump(logreg_smote, file)

print(f"Logistic Regression Accuracy: {accuracy_logreg}")
print(f"Logistic Regression Precision: {precision_logreg}")
print(f"Logistic Regression Recall: {recall_logreg}")
print(f"Logistic Regression F1 Score: {f1_logreg}")
print(str(confusion_matrix(y_test_smote, y_pred_logreg)))

Logistic Regression Accuracy: 0.6323097041607004
Logistic Regression Precision: 0.6316079645454739
Logistic Regression Recall: 0.6344150911246637
Logistic Regression F1 Score: 0.6330084157433906
[[177643 104238]
 [102986 178716]]


## Random Forest Classifier

In [ ]:
# Initialize and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42, n_estimators=50)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# save the model
with open("{}/rforest.pkl".format(path), "wb") as file:
    pickle.dump(rf_classifier, file)

print(f"Random Forest Accuracy: {accuracy_rf}")
print(f"Random Forest Precision: {precision_rf}")
print(f"Random Forest Recall: {recall_rf}")
print(f"Random Forest F1 Score: {f1_rf}")
print(str(confusion_matrix(y_test, y_pred_rf)))

Random Forest Accuracy: 0.7944784674353552
Random Forest Precision: 0.7682300802588904
Random Forest Recall: 0.8427993559282095
Random Forest F1 Score: 0.8037889343341272
[[ 95206  32372]
 [ 20014 107301]]


In [ ]:
# Initialize and train the Random Forest Classifier
rf_classifier_smote = RandomForestClassifier(random_state=42, n_estimators=50)
rf_classifier_smote.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred_rf = rf_classifier_smote.predict(X_test_smote)

# Evaluate the model
accuracy_rf = accuracy_score(y_test_smote, y_pred_rf)
precision_rf = precision_score(y_test_smote, y_pred_rf)
recall_rf = recall_score(y_test_smote, y_pred_rf)
f1_rf = f1_score(y_test_smote, y_pred_rf)

# save the model
with open("{}/rforest_smote.pkl".format(path), "wb") as file:
    pickle.dump(rf_classifier_smote, file)

print(f"Random Forest Accuracy: {accuracy_rf}")
print(f"Random Forest Precision: {precision_rf}")
print(f"Random Forest Recall: {recall_rf}")
print(f"Random Forest F1 Score: {f1_rf}")
print(str(confusion_matrix(y_test, y_pred_rf)))

Random Forest Accuracy: 0.8934478151399173
Random Forest Precision: 0.8905648142600887
Random Forest Recall: 0.8970614337136407
Random Forest F1 Score: 0.8938013189285164
[[250828  31053]
 [ 28998 252704]]


## XGBoost Classifier

In [ ]:
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

# Initialize and train the XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(
  learning_rate =0.1,
  n_estimators=1000,
  max_depth=5,
  min_child_weight=1,
  gamma=0,
  subsample=0.8,
  colsample_bytree=0.8,
  objective= 'binary:logistic',
  nthread=4,
  scale_pos_weight=1,
  seed=27,
  eval_metric = 'logloss')
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = xgb_classifier.predict(X_test)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

# save model
xgb_classifier.save_model("{}/xgb.model".format(path))

print(f"XGBoost Accuracy: {accuracy_xgb}")
print(f"XGBoost Precision: {precision_xgb}")
print(f"XGBoost Recall: {recall_xgb}")
print(f"XGBoost F1 Score: {f1_xgb}")
print(str(confusion_matrix(y_test, y_pred_xgb)))

XGBoost Accuracy: 0.6642787365678932
XGBoost Precision: 0.6693717234999107
XGBoost Recall: 0.6478733849114401
XGBoost F1 Score: 0.6584471204313864
[[86836 40742]
 [44831 82484]]


In [ ]:
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

# Initialize and train the XGBoost Classifier
xgb_classifier_smote = xgb.XGBClassifier(
  learning_rate =0.1,
  n_estimators=1000,
  max_depth=5,
  min_child_weight=1,
  gamma=0,
  subsample=0.8,
  colsample_bytree=0.8,
  objective= 'binary:logistic',
  nthread=4,
  scale_pos_weight=1,
  seed=27,
  eval_metric = 'logloss')
xgb_classifier_smote.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred_xgb = xgb_classifier_smote.predict(X_test_smote)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test_smote, y_pred_xgb)
precision_xgb = precision_score(y_test_smote, y_pred_xgb)
recall_xgb = recall_score(y_test_smote, y_pred_xgb)
f1_xgb = f1_score(y_test_smote, y_pred_xgb)

# save model
xgb_classifier_smote.save_model("{}/xgb_smote.model".format(path))

print(f"XGBoost Accuracy: {accuracy_xgb}")
print(f"XGBoost Precision: {precision_xgb}")
print(f"XGBoost Recall: {recall_xgb}")
print(f"XGBoost F1 Score: {f1_xgb}")
print(str(confusion_matrix(y_test_smote, y_pred_xgb)))

XGBoost Accuracy: 0.7776334630391619
XGBoost Precision: 0.8570971867007673
XGBoost Recall: 0.6662004529609303
XGBoost F1 Score: 0.749687413864044
[[250591  31290]
 [ 94032 187670]]


## One Class SVM Outlier Classifier

In [ ]:
#commenting out because this takes forever to run
"""
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)


# Split the resampled data
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

# Initialize and train the One-Class SVM
ocsvm = OneClassSVM(nu=0.1, kernel="rbf", gamma="scale")  # Adjust nu and kernel as needed
ocsvm.fit(X_train_resampled[y_train_resampled == 0]) # Train only on the negative class (no delay)


# Predict on the test set (all points will be either +1 (inlier) or -1(outlier))
y_pred_ocsvm = ocsvm.predict(X_test_resampled)


# Convert predictions to match the binary classification format (0, 1)
y_pred_ocsvm_binary = [0 if pred == 1 else 1 for pred in y_pred_ocsvm]


# Evaluate the model
accuracy_ocsvm = accuracy_score(y_test_resampled, y_pred_ocsvm_binary)
precision_ocsvm = precision_score(y_test_resampled, y_pred_ocsvm_binary)
recall_ocsvm = recall_score(y_test_resampled, y_pred_ocsvm_binary)
f1_ocsvm = f1_score(y_test_resampled, y_pred_ocsvm_binary)

with open("{}/ocsvm.pkl".format(path), "wb") as file:
    pickle.dump(ocsvm, file)

print(f"One-Class SVM Accuracy: {accuracy_ocsvm}")
print(f"One-Class SVM Precision: {precision_ocsvm}")
print(f"One-Class SVM Recall: {recall_ocsvm}")
print(f"One-Class SVM F1 Score: {f1_ocsvm}")
print(str(confusion_matrix(y_test_resampled, y_pred_ocsvm_binary)))
"""

'\nrus = RandomUnderSampler(random_state=42)\nX_resampled, y_resampled = rus.fit_resample(X, y)\n\n\n# Split the resampled data\nX_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(\n    X_resampled, y_resampled, test_size=0.2, random_state=42\n)\n\n# Initialize and train the One-Class SVM\nocsvm = OneClassSVM(nu=0.1, kernel="rbf", gamma="scale")  # Adjust nu and kernel as needed\nocsvm.fit(X_train_resampled[y_train_resampled == 0]) # Train only on the negative class (no delay)\n\n\n# Predict on the test set (all points will be either +1 (inlier) or -1(outlier))\ny_pred_ocsvm = ocsvm.predict(X_test_resampled)\n\n\n# Convert predictions to match the binary classification format (0, 1)\ny_pred_ocsvm_binary = [0 if pred == 1 else 1 for pred in y_pred_ocsvm]\n\n\n# Evaluate the model\naccuracy_ocsvm = accuracy_score(y_test_resampled, y_pred_ocsvm_binary)\nprecision_ocsvm = precision_score(y_test_resampled, y_pred_ocsvm_binary)\nrecall_ocsvm = recall_

## Neural Network

In [ ]:
# Parameters
num_epochs = 100
learning_rate = 0.0005
hidden_units = 64
decay = 0.001


x_tensor = torch.FloatTensor(x)
y_tensor = torch.FloatTensor(y)

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_tensor, y_tensor, test_size=0.2, random_state=42)

# Define the neural network architecture
class weatherNN(nn.Module):
    def __init__(self):
        super(weatherNN, self).__init__()
        self.fc1 = nn.Linear(8, hidden_units)
        self.fc2 = nn.Linear(hidden_units, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = weatherNN()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(x_train)
    loss = criterion(outputs.squeeze(), y_train)

    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_probs = model(x_test)
    y_pred = (y_pred_probs.squeeze() > 0.5).float()

# Calculate metrics
accuracy = accuracy_score(y_test.numpy(), y_pred.numpy())
precision = precision_score(y_test.numpy(), y_pred.numpy())
recall = recall_score(y_test.numpy(), y_pred.numpy())
f1 = f1_score(y_test.numpy(), y_pred.numpy())
conf_matrix = confusion_matrix(y_test.numpy(), y_pred.numpy())

# Print the results
print(f'Neural Network Accuracy: {accuracy:.4f}')
print(f'Neural Network Precision: {precision:.4f}')
print(f'Neural Network Recall: {recall:.4f}')
print(f'Neural Network F1 Score: {f1:.4f}')
print(conf_matrix)

Neural Network Accuracy: 0.5605
Neural Network Precision: 0.5602
Neural Network Recall: 0.5588
Neural Network F1 Score: 0.5595
[[71718 55860]
 [56166 71149]]
